In [15]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import pandas as pd
import time
import copy

tan = np.tan
cos = np.cos
sin = np.sin
pi = np.pi
taninv = np.arctan
transpose = np.transpose

In [16]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import pandas as pd
import time
import copy
import coefficients
import kinematics
import transformations


tan = np.tan
cos = np.cos
sin = np.sin
pi = np.pi
transpose = np.transpose

### PARAMETERS ###
c = 0.0488 # chord
R = 0.269 # Radius
S = 0.228 # Surface Area
m = 0.13 # mass
rho = 1.225
g = 9.8

length_1 = 0.3
length_2 = 0.3
theta_1 = 0.
theta_2 = 0.
Lamda_1 = pi*2/3
Lamda_2 = pi*4/3
x_ac_1 = 0.0723
x_ac_2 = 0.0723
beta_1 = 0.
beta_2 = 0.
segments = 100

inertia_jframe = np.array([[1.9*10**-3, 0., 0.],[0.,4.88*10**-6, 0.],[0.,0.,1.9*10**-3]])
x_ac_vec = np.array([x_ac_1,x_ac_2])
length_vec = np.array([length_1, length_2]) # length of blades
theta_pitch_vec = np.array([theta_1, theta_2]) # theta_pitch of blades
Lamda_vec = np.array([Lamda_1,Lamda_2]) # Lamda of blades
beta_vec = np.array([0.,0.]) # beta of blades

delta_t = 0.2 # time step

# Initial Conditions
u_vec = np.array([10.,0.,0.]) # initialize u_vec
omega_vec = np.array([0.,0.,1.]) # initialize omega_vec
u_vec_d = np.zeros(3) # dummy initialisation
Omega = omega_vec[2] # r at t = 0
path_vec = np.array([0.,0.,1]) # initial X0,Y0,Z0
phi = 60.*pi/180.
theta = 0.
psi = 0.
Phi0 = 60.*pi/180.
Theta0 = 0.
Psi0 = 0.
Phi = 60.*pi/180.
Psi = 0.
Theta = 0.
lamda = 0.
### END PARAMETERS ####

### OVERALL TRAJECTORY ####
trajectory_u_vec = []
trajectory_omega_vec = []
trajectory_Phi_Theta_Psi = []
trajectory_path = []
trajectory_phi_theta_psi = []

# Unchanged params
Tj1 = transformations.doT0TjTransformation(Lamda_vec[0],theta_pitch_vec[0],beta_vec[0]) # blade 1
Tj2 = transformations.doT0TjTransformation(Lamda_vec[1],theta_pitch_vec[1],beta_vec[1]) # blade 2

In [32]:
r_j_vec_1 = kinematics.doPositionVector(Tj1,length_vec[0],x_ac_vec[0])
r_j_vec_2 = kinematics.doPositionVector(Tj2,length_vec[1],x_ac_vec[1])

J1 = np.matmul(transpose(Tj1),inertia_jframe)
J2 = np.matmul(transpose(Tj2),inertia_jframe)

# initial velocity
trajectory_u_vec.append(copy.copy(u_vec))
trajectory_omega_vec.append(copy.copy(omega_vec))
trajectory_Phi_Theta_Psi.append(copy.copy(np.array([Phi, Theta, Psi])))

a = time.time()
steps = 0
while steps <= 1 :
    ## Changes over time ###

    # kinematics
    v_vec_1 = kinematics.doVelBlade(u_vec, omega_vec, r_j_vec_1)
    v_vec_2 = kinematics.doVelBlade(u_vec, omega_vec, r_j_vec_2)

    w_vec_1 = kinematics.doRelativeAirVelBlade(v_vec_1,Tj1)
    w_vec_2 = kinematics.doRelativeAirVelBlade(v_vec_2,Tj2)
    
    alpha_vec_1 = kinematics.doAlpha(w_vec_1)
    alpha_vec_2 = kinematics.doAlpha(w_vec_2)
    
    # Coefficients - Aerodynamic and Body
    Cx_A = coefficients.doCx_A(w_vec_1, c, R, S, Omega, alpha_vec_1, Lamda_1, theta_1, beta_1,  length_1, segments)
    Cx_A += coefficients.doCx_A(w_vec_2, c, R, S, Omega, alpha_vec_2, Lamda_2, theta_2, beta_2,  length_2, segments)


    steps+=1

print(alpha_vec_1)

AttributeError: 'numpy.float64' object has no attribute 'append'

In [29]:
alpha_vec = []
w_vec = [[1,0,1],[1,0,1]]
def doAlpha(w_vec) :
    alpha_vec = []
    for i in w_vec :
        if i[2]>=0 and i[0]>=0 :
            alpha = pi - taninv(abs(i[2]/i[0]))
        elif i[2]<=0 and i[0]>=0 :
            alpha = taninv(abs(i[2]/i[0])) - pi
        elif i[2]<=0 and i[0]<=0 :
            alpha = -taninv(abs(i[2]/i[0]))
        else :
            alpha = taninv(abs(i[2]/i[0]))
        alpha_vec.append(alpha)
    return alpha_vec


In [30]:
al = doAlpha(w_vec)